## Problème à patch
Problème majeur, le stockage de l'écart type est fait en floatant hors l'écart entre les écarts types est inférieur à un floatant
stockage à changer sous forme d'entier plus précis

In [2]:
def lecture_csv(col:str="", limit:int=float('infinity')):
    """
    Lis le fichier loto.csv puis crée un dictionnaire de répartition de chaque boule allant de 1 à @limit
    """
    import pandas as pd;
    df = pd.read_csv("loto.csv", sep=';')


    # Creation du set de chaque numero_chance
    numero_chance:dict[int, int] = {i: 0 for i in range(1, min(10, limit)+1)}
    # Initialisation du nombre de numero_chance
    for each in df["numero_chance"]:
        if each <= limit:
            numero_chance[each] += 1

    # Creation du set de chaque boule
    boule:dict[int, int] = {i: 0 for i in range(1, min(49, limit)+1)}
    # Initialisation du nombre de boule
    for i in range(1, 5):
        # Pour les 5 sets
        for each in df[f"boule_{i}"]:
            if each <= limit:
                boule[each] += 1

    # Décision du retour
    d, size = (numero_chance, min(10, limit)) if col == "chance" else (boule, min(49, limit))
    return (d, sum(d.values()), size)

In [3]:
def show(d:dict[int,int], t:int) -> dict[int,float]:
    # Return the percrentage of apparition of each balls
    return {key: val/t for key, val in d.items()}

In [4]:
def moy_a_16(d:dict[int, int], size:int) -> int:
    normal:float = 1/size
    # Retourne l'écart type des pourcentages
    return sum([abs(_ - normal) for _ in d.values()])

### Ancien ecart type

In [9]:
def ecart_type_min(d:dict[int, int], t:int, size:int) -> list[tuple[int, float]]:
    # Initialisation du set de pourcentage
    percent:dict[int, int] = {}
    
    # On rajoute un seul tirage
    t += 1

    
    # Pour chaque numero de boule
    for i in range(1, size+1):
        # On copie le set d'origine
        data:dict[int, int] = d.copy()
        # Puis on ajoute le tirage souhaité à l'interieur
        data[i] += 1
        # On calcule le nouvel écart type et l'ajoute au pourcentage
        percent[i]:int = moy_a_16(show(data, t), size)
    return percent

### Nouvel écart type

In [6]:
def ecart_type_min(d:dict[int, int]={}, t:int=0, size:int=0) -> list[tuple[int, float]]:
    # Initialisation du set de pourcentage
    percent = {}

    # On rajoute un seul tirage
    t += 1

    # Pour chaque numero de boule
    for i in range(1, size+1):
        print(i)
        # On copie le set d'origine
        data_b1 = d.copy()
        # Puis on ajoute le tirage souhaité à l'interieur
        data_b1[i] += 1

        for j in range(1, size+1):
            if j == i:continue

            data_b2 = data_b1.copy()
            data_b2[j] += 1
            for k in range(1, size+1):
                if k == i or k == j:continue

                data_b3 = data_b2.copy()
                data_b3[k] += 1
                for l in range(1, size+1):
                    if l == k or l == j or l == i:continue

                    data_b4 = data_b3.copy()
                    data_b4[l] += 1
                    for m in range(1, size+1):
                        if m == l or m == k or m == j or m == i:continue

                        data_b5 = data_b4.copy()
                        data_b5[m] += 1

                        # On calcule le nouvel écart type et l'ajoute au pourcentage
                        percent[(i, j, k, l, m)] = moy_a_16(show(data_b5, t), size)
        
    return percent

In [7]:
def result(d:dict[int, int], t:int, size:int):
    # Retourne la liste triée des écarts types
    s = sorted(ecart_type_min(d, t, size).items(), key=lambda x: x[1])
    #[print(each) for each in s]
    return s

In [10]:
data, times, size = lecture_csv()
result(data, times, size)

[(1, 0.09090909090909091),
 (7, 0.09090909090909091),
 (3, 0.09090909090909093),
 (5, 0.09090909090909093),
 (6, 0.09090909090909093),
 (8, 0.09090909090909093),
 (2, 0.09393939393939395),
 (4, 0.09393939393939395),
 (9, 0.09393939393939395),
 (10, 0.09393939393939396)]